In [47]:
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
import os
import pandas as pd

# Path ke folder yang berisi file-file DataFrame
folder_path = '../comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/train'

# Dictionary untuk menyimpan semua DataFrame dengan nama file sebagai key
df_dict = {}

# Loop melalui semua file dalam folder
for file_name in os.listdir(folder_path):
    # Pastikan file yang dibaca adalah file yang diinginkan (misalnya, CSV)
    if file_name.endswith('.csv'):
        # Buat path lengkap ke file
        file_path = os.path.join(folder_path, file_name)
        
        # Baca file dan simpan ke dictionary dengan nama file sebagai key
        df = pd.read_csv(file_path)
        df_dict[file_name] = df  # Gunakan nama file sebagai key


In [49]:
df_bawang_merah = df_dict['Bawang Merah.csv']
df_bawang_putih = df_dict['Bawang Putih Bonggol.csv']
df_beras_medium = df_dict['Beras Medium.csv']
df_beras_premium = df_dict['Beras Premium.csv']
df_cabai_merah_keriting = df_dict['Cabai Merah Keriting.csv']
df_cabai_rawit_merah = df_dict['Cabai Rawit Merah.csv']
df_daging_ayam = df_dict['Daging Ayam Ras.csv']
df_daging_sapi = df_dict['Daging Sapi Murni.csv']
df_gula = df_dict['Gula Konsumsi.csv']
df_minyak_goreng_curah = df_dict['Minyak Goreng Curah.csv']
df_minyak_goreng_kemasan = df_dict['Minyak Goreng Kemasan Sederhana.csv']
df_telur_ayam = df_dict['Telur Ayam Ras.csv']
df_tepung_terigu = df_dict['Tepung Terigu (Curah).csv']

In [50]:
import pandas as pd

# List semua dataframe
df_list = [
    df_bawang_merah, df_bawang_putih, df_beras_medium, df_beras_premium, df_cabai_merah_keriting,
    df_cabai_rawit_merah, df_daging_ayam, df_daging_sapi, df_gula, df_minyak_goreng_curah,
    df_minyak_goreng_kemasan, df_telur_ayam, df_tepung_terigu
]

# Loop setiap dataframe dan lakukan interpolasi
for i in range(len(df_list)):
    df = df_list[i]  # Ambil dataframe

    # Pisahkan kolom tanggal (asumsi ada kolom 'date' atau 'tanggal')
    date_cols = df.select_dtypes(include=['datetime64', 'object'])  # Deteksi kolom tanggal
    num_cols = df.select_dtypes(include=['number'])  # Ambil hanya kolom numerik

    # Interpolasi hanya untuk kolom numerik
    df_num_imputed = num_cols.interpolate(method='linear', limit_direction='both')

    # Gabungkan kembali kolom tanggal dengan hasil interpolasi
    df_list[i] = pd.concat([date_cols.reset_index(drop=True), df_num_imputed.reset_index(drop=True)], axis=1)


In [51]:
df_dict = {
    'Bawang Merah': df_list[0],
    'Bawang Putih Bonggol': df_list[1],
    'Beras Medium': df_list[2],
    'Beras Premium': df_list[3],
    'Cabai Merah Keriting': df_list[4],
    'Cabai Rawit Merah': df_list[5],
    'Daging Ayam Ras': df_list[6],
    'Daging Sapi Murni': df_list[7],
    'Gula Konsumsi': df_list[8],
    'Minyak Goreng Curah': df_list[9],
    'Minyak Goreng Kemasan Sederhana': df_list[10],
    'Telur Ayam Ras': df_list[11],
    'Tepung Terigu (Curah)': df_list[12]
}


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Parameter
seq_length = 30  # Panjang sequence
learning_rate = 0.001
epochs = 100
# Rentang waktu prediksi
predict_start = '2024-10-01'
predict_end = '2024-12-31'
predict_periods = (pd.to_datetime(predict_end) - pd.to_datetime(predict_start)).days + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [53]:
class BiLSTMPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=3, output_size=1, dropout=0.3):
        super(BiLSTMPredictor, self).__init__()
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout, bidirectional=True)
        
        # Fully connected layer (hidden_size * 2 because of bidirection)
        self.fc = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)  
        out = self.fc(lstm_out[:, -1, :])  # Take last time step
        return out.unsqueeze(-1)  # Ensures output shape (batch_size, 1, 1)

In [54]:
# Direktori penyimpanan model terbaik
if not os.path.exists("best_models"): os.makedirs("best_models")

# Menyimpan hasil prediksi dan MAPE terbaik
all_predictions = []
mape_results = {}

In [55]:
for komoditas, df_komoditas in df_dict.items():
    print(f"\n=== Memproses {komoditas} ===")

    df_komoditas['Date'] = pd.to_datetime(df_komoditas['Date'])
    
    for provinsi in df_komoditas.columns[1:]:
        print(f"  → Prediksi untuk {provinsi}...")

        df_prov = df_komoditas[['Date', provinsi]].rename(columns={'Date': 'ds', provinsi: 'y'})
        
        # Normalisasi data
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df_prov[['y']])
        
        X, y = [], []
        for i in range(len(df_scaled) - seq_length):
            X.append(df_scaled[i:i+seq_length])  # Menggunakan hanya harga komoditas sebagai input
            y.append(df_scaled[i+seq_length])  # Target harga komoditas
        
        X, y = np.array(X), np.array(y)
        X = torch.tensor(X, dtype=torch.float32).to(device)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1).to(device)
        
        # Pisahkan training & testing
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Bangun model LSTM
        model = BiLSTMPredictor(input_size=1, hidden_size=128, num_layers=3, output_size=1, dropout=0.3).to(device)
        criterion = nn.SmoothL1Loss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
        
        best_val_loss = float("inf")
        best_mape = float("inf")
        best_model_path = f"best_models/univariate_{komoditas}_{provinsi}.pt"

        # Train model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 10 == 0:
                model.eval()
                with torch.no_grad():
                    test_output = model(X_test)
                    val_loss = criterion(test_output, y_test).item()
                    
                    test_output_np = test_output.cpu().numpy().reshape(-1, 1)
                    y_test_np = y_test.cpu().numpy().reshape(-1, 1)

                    test_output_np = scaler.inverse_transform(test_output_np)
                    y_test_np = scaler.inverse_transform(y_test_np)

                    mape = mean_absolute_percentage_error(y_test_np, test_output_np)


                print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item():.6f}, Validation Loss: {val_loss:.6f}, MAPE: {mape:.6f}")
                
                scheduler.step(val_loss)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_mape = mape
                    torch.save(model.state_dict(), best_model_path)
        
        mape_results[f"{komoditas}_{provinsi}"] = best_mape

        # Load model terbaik untuk prediksi masa depan
        best_model = BiLSTMPredictor(input_size=1).to(device)
        best_model.load_state_dict(torch.load(best_model_path))
        best_model.eval()
        
        last_sequence = X[-1].unsqueeze(0)
        future_preds = []

        for _ in range(predict_periods):
            with torch.no_grad():
                next_pred = best_model(last_sequence).item()
            future_preds.append(next_pred)
            
            next_input = torch.tensor([[next_pred]], dtype=torch.float32).unsqueeze(0).to(device)
            last_sequence = torch.cat([last_sequence[:, 1:, :], next_input], dim=1)

        future_preds = np.array(future_preds).reshape(-1, 1)
        future_preds = scaler.inverse_transform(future_preds)

        for i, pred in enumerate(future_preds):
            date = pd.to_datetime(predict_start) + pd.Timedelta(days=i)
            all_predictions.append({
                'id': f"{komoditas}/{provinsi}/{date.strftime('%Y-%m-%d')}",
                'price': pred[0]
            })

# Simpan hasil prediksi ke CSV
submission_df = pd.DataFrame(all_predictions)
submission_df.to_csv('submission_univariate_bilstm_mape.csv', index=False)


=== Memproses Bawang Merah ===
  → Prediksi untuk Aceh...


/home/sirius/Documents/DS/myenv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 10/100, Training Loss: 0.013335, Validation Loss: 0.058884, MAPE: 0.260016
Epoch 20/100, Training Loss: 0.012465, Validation Loss: 0.048769, MAPE: 0.251733
Epoch 30/100, Training Loss: 0.010238, Validation Loss: 0.043131, MAPE: 0.224111
Epoch 40/100, Training Loss: 0.002003, Validation Loss: 0.006182, MAPE: 0.079406
Epoch 50/100, Training Loss: 0.001690, Validation Loss: 0.005124, MAPE: 0.079443
Epoch 60/100, Training Loss: 0.000971, Validation Loss: 0.002363, MAPE: 0.048212
Epoch 70/100, Training Loss: 0.000846, Validation Loss: 0.002225, MAPE: 0.044685
Epoch 80/100, Training Loss: 0.000657, Validation Loss: 0.001852, MAPE: 0.040653
Epoch 90/100, Training Loss: 0.000614, Validation Loss: 0.001653, MAPE: 0.037235
Epoch 100/100, Training Loss: 0.000600, Validation Loss: 0.001705, MAPE: 0.038738
  → Prediksi untuk Bali...


/tmp/ipykernel_12205/2901808583.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv/

Epoch 10/100, Training Loss: 0.013268, Validation Loss: 0.023851, MAPE: 0.210567
Epoch 20/100, Training Loss: 0.012013, Validation Loss: 0.023430, MAPE: 0.201997
Epoch 30/100, Training Loss: 0.010223, Validation Loss: 0.019516, MAPE: 0.187919
Epoch 40/100, Training Loss: 0.005104, Validation Loss: 0.005801, MAPE: 0.123611
Epoch 50/100, Training Loss: 0.001720, Validation Loss: 0.002904, MAPE: 0.063917
Epoch 60/100, Training Loss: 0.001085, Validation Loss: 0.001514, MAPE: 0.048599
Epoch 70/100, Training Loss: 0.000924, Validation Loss: 0.001391, MAPE: 0.044732
Epoch 80/100, Training Loss: 0.000821, Validation Loss: 0.001115, MAPE: 0.041386
Epoch 90/100, Training Loss: 0.000761, Validation Loss: 0.001069, MAPE: 0.039317
Epoch 100/100, Training Loss: 0.000668, Validation Loss: 0.000954, MAPE: 0.037973
  → Prediksi untuk Banten...


...

/tmp/ipykernel_12205/2901808583.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv/

Epoch 10/100, Training Loss: 0.035280, Validation Loss: 0.052283, MAPE: 0.107684
Epoch 20/100, Training Loss: 0.013853, Validation Loss: 0.028253, MAPE: 0.078024
Epoch 30/100, Training Loss: 0.011559, Validation Loss: 0.020269, MAPE: 0.065489
Epoch 40/100, Training Loss: 0.009862, Validation Loss: 0.030960, MAPE: 0.082494
Epoch 50/100, Training Loss: 0.006684, Validation Loss: 0.011914, MAPE: 0.050486
Epoch 60/100, Training Loss: 0.000895, Validation Loss: 0.000664, MAPE: 0.010380
Epoch 70/100, Training Loss: 0.000838, Validation Loss: 0.001829, MAPE: 0.019539
Epoch 80/100, Training Loss: 0.000590, Validation Loss: 0.000581, MAPE: 0.010252
Epoch 90/100, Training Loss: 0.000578, Validation Loss: 0.000312, MAPE: 0.006576
Epoch 100/100, Training Loss: 0.000549, Validation Loss: 0.000650, MAPE: 0.010976
  → Prediksi untuk Sumatera Utara...


/tmp/ipykernel_12205/2901808583.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv/

Epoch 10/100, Training Loss: 0.023710, Validation Loss: 0.012182, MAPE: 0.041248
Epoch 20/100, Training Loss: 0.018589, Validation Loss: 0.006508, MAPE: 0.028906
Epoch 30/100, Training Loss: 0.014636, Validation Loss: 0.013280, MAPE: 0.043319
Epoch 40/100, Training Loss: 0.002109, Validation Loss: 0.001564, MAPE: 0.010833
Epoch 50/100, Training Loss: 0.000774, Validation Loss: 0.001413, MAPE: 0.011727
Epoch 60/100, Training Loss: 0.000939, Validation Loss: 0.001128, MAPE: 0.009789
Epoch 70/100, Training Loss: 0.000679, Validation Loss: 0.001014, MAPE: 0.008517
Epoch 80/100, Training Loss: 0.000612, Validation Loss: 0.001006, MAPE: 0.008022
Epoch 90/100, Training Loss: 0.000563, Validation Loss: 0.001036, MAPE: 0.008876
Epoch 100/100, Training Loss: 0.000594, Validation Loss: 0.000998, MAPE: 0.008150


/tmp/ipykernel_12205/2901808583.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))


In [56]:
avg_mape = np.mean(list(mape_results.values()))
print(f"\nRata-rata MAPE dari model terbaik: {avg_mape:.6f}")


Rata-rata MAPE dari model terbaik: 0.019095
